In [0]:
# Import the libraries and data.
import os
import yaml
import datetime
from datetime import date
import numpy as np
import pandas as pd
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80)

#import plotly.offline as py
#py.init_notebook_mode()

# Did some pre processing of the CSV. 
# Specficially I:
#   - truncated a bunch of notes at the top of the file
 #   - removed some whitespace at the end of file

from google.colab import files
uploaded = files.upload()

#df = pd.read_csv('moz_2013_2016.csv', sep = ',', thousands=',')
#print(df.dtypes)

In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['moz-forecasting.csv']))
# Dataset is now stored in a Pandas Dataframe


In [0]:
df.head()

In [0]:
print(df.dtypes)

In [0]:
#data cleanup
df['ds'] = df['Ds']
df['y'] = df['Y']
#drop extra columns
df = df[['ds', 'y']]

In [0]:
df.head()

In [0]:
# Convert to a date time
#df['ds'] = df['ds'].astype('datetime64[ns]')
df['ds'] = pd.to_datetime(df['ds'])

In [0]:
# Make model and fit it
m = Prophet()
m.fit(df)

In [0]:

# Make a future data frame
future = m.make_future_dataframe(periods=30)
future.tail()

In [0]:
# Predict the GA data over the future period
forecast = m.predict(future)
#forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
forecast.tail(30)

In [0]:
# plot the forecast
fig1 = m.plot(forecast)

In [0]:
fig2 = m.plot_components(forecast)

In [0]:
forecast_date = '07-01-2019'


In [0]:
# Filter down to only data before the start date
mask = (df['ds'] < forecast_date)
df2 = df.loc[mask]
df2.head()

In [0]:
df

In [0]:

# Make model and fit it
m2 = Prophet()
m2.fit(df2)

# Make a future data frame
future = m2.make_future_dataframe(periods=90)
future.tail()

# Predict the GA data over the future period
forecast = m2.predict(future)
forecast.tail()
#forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [0]:
# plot the forecast
fig2 = m2.plot(forecast)

In [0]:
# Merge actuals with forecast
forecast_plot = forecast[['ds', 'yhat']]
df_inner = pd.merge(forecast_plot, df, on='ds', how='inner')
df_inner.tail()

In [0]:

mask = (df_inner['ds'] > forecast_date)
df2_plot = df_inner.loc[mask]
df2_plot.tail()

In [0]:
# Plot actuals vs forecast
plt.figure(figsize=(16, 9))
plt.title(label='Forecast vs. Actual Performance \n' + 'forecast date = ' + forecast_date)
plt.plot('ds', 'y', data=df2_plot, color='black')
plt.plot('ds', 'yhat', data=df2_plot, color ='skyblue', linestyle='dashed')
plt.show()